In [5]:
import os
import numpy as np
from tqdm import tqdm
import pandas as pd
import pandapower as pp
import pandapower.networks as pn
import pandapower.plotting as plot
from pandapower.control import ConstControl
from pandapower.timeseries import DFData, OutputWriter, run_timeseries
import matplotlib.pyplot as plt
from scipy.stats import norm

In [9]:
# Define criteria thresholds
voltage_threshold = 0.95  # Minimum acceptable voltage in p.u.
line_loading_threshold = 100.0  # Maximum acceptable line loading in percentage

# Load the 30-bus system
net = pn.case30()
# Perform base case power flow
pp.runpp(net)
# Check the base case results
print("Base Case: Power flow successfully converged.")
base_voltages = net.res_bus['vm_pu']
base_line_loadings = net.res_line['loading_percent']

Base Case: Power flow successfully converged.


In [ ]:
# List of lines to consider for contingencies
line_contingencies = net.line.index.tolist()

# List of generators to consider for contingencies
gen_contingencies = net.gen.index.tolist()

In [14]:
# Results storage
contingency_results = []

# Loop through line contingencies
for line in line_contingencies:
    # Backup the original line state
    original_in_service = net.line.at[line, 'in_service']
    
    # Simulate the contingency by opening the line
    net.line.at[line, 'in_service'] = False
    
    try:
        # Run power flow
        pp.runpp(net)
        
        # Check system stability (voltage and line loading limits)
        max_voltage = net.res_bus['vm_pu'].max()
        min_voltage = net.res_bus['vm_pu'].min()
        max_loading = net.res_line['loading_percent'].max()
        
        contingency_results.append({
            'Contingency': f'Line {line}',
            'Max Voltage (p.u.)': max_voltage,
            'Min Voltage (p.u.)': min_voltage,
            'Max Line Loading (%)': max_loading,
            'Status': 'Stable' if max_voltage < 1.1 and min_voltage > 0.9 and max_loading < 100 else 'Unstable'
        })
    except pp.LoadflowNotConverged:
        contingency_results.append({
            'Contingency': f'Line {line}',
            'Max Voltage (p.u.)': None,
            'Min Voltage (p.u.)': None,
            'Max Line Loading (%)': None,
            'Status': 'Failed to converge'
        })
    
    # Restore the original line state
    net.line.at[line, 'in_service'] = original_in_service

# Loop through generator contingencies
for gen in gen_contingencies:
    # Backup the original generator state
    original_in_service = net.gen.at[gen, 'in_service']
    
    # Simulate the contingency by shutting down the generator
    net.gen.at[gen, 'in_service'] = False
    
    try:
        # Run power flow
        pp.runpp(net)
        
        # Check system stability (voltage and line loading limits)
        max_voltage = net.res_bus['vm_pu'].max()
        min_voltage = net.res_bus['vm_pu'].min()
        max_loading = net.res_line['loading_percent'].max()
        
        contingency_results.append({
            'Contingency': f'Generator {gen}',
            'Max Voltage (p.u.)': max_voltage,
            'Min Voltage (p.u.)': min_voltage,
            'Max Line Loading (%)': max_loading,
            'Status': 'Stable' if max_voltage < 1.1 and min_voltage > 0.9 and max_loading < 100 else 'Unstable'
        })
    except pp.LoadflowNotConverged:
        contingency_results.append({
            'Contingency': f'Generator {gen}',
            'Max Voltage (p.u.)': None,
            'Min Voltage (p.u.)': None,
            'Max Line Loading (%)': None,
            'Status': 'Failed to converge'
        })
    
    # Restore the original generator state
    net.gen.at[gen, 'in_service'] = original_in_service


In [15]:
df_results = pd.DataFrame(contingency_results)
df_results.to_excel("n1_contingency_results.xlsx", index=False)
print("Contingency results saved to 'n1_contingency_results.xlsx'.")

Contingency results saved to 'n1_contingency_results.xlsx'.


In [13]:
import numpy as np
import pandas as pd
import pandapower as pp
import pandapower.networks as pn

# Load the 30-bus system
net = pn.case30()

# Define PV buses
pv_buses = [1, 21, 26, 22, 12]

# Fixed means and standard deviations for injected power
pv_bus_means = [20.42496, 12.76560, 14.04216, 7.65936, 10.21248]
pv_bus_stds = [25.622111, 16.013819, 17.615201, 9.608292, 12.811055]

# Number of scenarios and Monte Carlo runs per scenario
n_scenarios = 10  # For demonstration purposes, use fewer scenarios
n_runs_per_scenario = 100

# Results storage
scenario_results = []

# Function to calculate whether N-1 is satisfied
def n1_check(net, contingency_type, contingency_index):
    try:
        if contingency_type == "line":
            # Open the line
            net.line.at[contingency_index, 'in_service'] = False
        elif contingency_type == "generator":
            # Turn off the generator
            net.gen.at[contingency_index, 'in_service'] = False
        
        # Run power flow
        pp.runpp(net)
        
        # Check for stability
        max_voltage = net.res_bus['vm_pu'].max()
        min_voltage = net.res_bus['vm_pu'].min()
        max_loading = net.res_line['loading_percent'].max()
        
        # Restore the contingency component
        if contingency_type == "line":
            net.line.at[contingency_index, 'in_service'] = True
        elif contingency_type == "generator":
            net.gen.at[contingency_index, 'in_service'] = True

        # Criterion for N-1 satisfaction: No voltage violation and no overload
        return max_voltage < 1.1 and min_voltage > 0.9 and max_loading < 100
    except pp.LoadflowNotConverged:
        # If power flow does not converge, the N-1 criterion is not satisfied
        return False

# Generate scenarios
for scenario in range(n_scenarios):
    print(f"Running Scenario {scenario + 1}/{n_scenarios}")
    lolp_count = 0
    n1_failures = 0  # Count N-1 violations
    total_solar_generation = []
    
    for run in range(n_runs_per_scenario):
        # Sample PV outputs from normal distribution
        pv_outputs = [max(0, np.random.normal(mean, std)) for mean, std in zip(pv_bus_means, pv_bus_stds)]
        
        # Update PV generation in the network
        for i, bus in enumerate(pv_buses):
            if bus in net.sgen['bus'].values:
                net.sgen.loc[net.sgen['bus'] == bus, 'p_mw'] = pv_outputs[i]
            else:
                pp.create_sgen(net, bus, p_mw=pv_outputs[i], name=f"PV_bus_{bus}")
        
        try:
            # Run power flow
            pp.runpp(net)
            
            # Calculate metrics
            total_generation = net.res_sgen['p_mw'].sum() + net.res_gen['p_mw'].sum()
            total_load = net.load['p_mw'].sum()
            
            # LOLP: 1 if generation < load
            if total_generation < total_load:
                lolp_count += 1
            
            # N-1 Check for lines
            for line_index in net.line.index:
                if not n1_check(net, contingency_type="line", contingency_index=line_index):
                    n1_failures += 1
            
            # N-1 Check for generators
            for gen_index in net.gen.index:
                if not n1_check(net, contingency_type="generator", contingency_index=gen_index):
                    n1_failures += 1

        except pp.LoadflowNotConverged:
            print(f"Power flow did not converge for scenario {scenario + 1}, run {run + 1}.")
            continue
    
    # Calculate average LOLP and N-1 failure rate
    avg_lolp = lolp_count / n_runs_per_scenario
    n1_failure_rate = n1_failures / (n_runs_per_scenario * (len(net.line) + len(net.gen)))
    
    # Record scenario results
    scenario_results.append({
        'Scenario': scenario + 1,
        'Average LOLP': avg_lolp,
        'N-1 Failure Rate': n1_failure_rate
    })

# Save results to a DataFrame
df_scenario_results = pd.DataFrame(scenario_results)
df_scenario_results.to_excel("n1_solar_scenarios.xlsx", index=False)
print("Scenario results saved to 'n1_solar_scenarios.xlsx'.")


Running Scenario 1/10
Running Scenario 2/10
Running Scenario 3/10
Running Scenario 4/10
Running Scenario 5/10
Running Scenario 6/10
Running Scenario 7/10
Running Scenario 8/10
Running Scenario 9/10
Running Scenario 10/10
Scenario results saved to 'n1_solar_scenarios.xlsx'.
